<a id='back'></a>

### Оглавление

1. [Описание задачи](#task)


2. [Описание датасета](#dataset)


3. [Решение задачи](#solution)

    3.1 [Шаг 1. Загрузить данные и изучить общую информацию](#solution_1)
    
    3.2 [Шаг 2. Выполнить предобработку данных](#solution_2) 
    
    3.3 [Шаг 3. Провести анализ предоставленных данных](#solution_3) 
    
    3.4 [Шаг 4. Провести детализацию исследования: открытие кофейни](#solution_4) 

<a id='task'></a>

### Описание задачи

Необходимо подготовить исследование рынка заведений общественного питания Москвы с целью открытия нового бизнеса - места общественного питания: кафе, ресторана, пиццерии, паба или бара. Также необходимо определить, какими будут расположение, меню и цены заведения для максимизации прибыли с заведения.

Пользователи отчета - инвесторы из фонда «Shut Up and Take My Money».

Решение данной задачи будет представлено в виде отчета в Jupyter Notebook и презентации на основании данного отчета с основными тезисами и выводами. 

Решение в отчете содержит следующие шаги:

Шаг 1. Загрузить данные и изучить общую информацию;

Шаг 2. Выполнить предобработку данных;

Шаг 3. Провести анализ предоставленных данных;

Шаг 4. Провести детализацию исследования: открытие кофейни;

Шаг 5. Подготовить презентацию

**Презентация:** https://disk.yandex.ru/i/4e8R7KJnp8_BJw 

<a id='dataset'></a>

### Описание датасета 

Вам доступен датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года. 

Информация, размещённая в сервисе Яндекс Бизнес, могла быть добавлена пользователями или найдена в общедоступных источниках. Она носит исключительно справочный характер.

Файл *moscow_places.csv:*
* name — название заведения;
* address — адрес заведения;
* category — категория заведения, например «кафе», «пиццерия» или «кофейня»;
* hours — информация о днях и часах работы;
* lat — широта географической точки, в которой находится заведение;
* lng — долгота географической точки, в которой находится заведение;
* rating — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
* price — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;
* avg_bill — строка, которая хранит среднюю стоимость заказа в виде диапазона;
* middle_avg_bill — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»
* middle_coffee_cup — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»
* chain — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):
    * 0 — заведение не является сетевым
    * 1 — заведение является сетевым
* district — административный район, в котором находится заведение, например Центральный административный округ;
* seats — количество посадочных мест.

[Вернуться наверх](#back)

<a id='solution'></a>
### Решение

In [ ]:
#импортируем библиотеки

import pandas as pd
import matplotlib.pyplot as plt
from plotly import graph_objects as go 
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

# подключаем модуль для работы с JSON-форматом
import json
import folium
from folium import Map, Choropleth, Marker

# импортируем кластер
from folium.plugins import MarkerCluster

# импортируем собственные иконки
from folium.features import CustomIcon

[Вернуться наверх](#back)

<a id='solution_1'></a>

#### Шаг 1. Загрузить данные и изучить общую информацию;

In [ ]:
#подгружаем данные 

try: 
    data = pd.read_csv('~/Desktop/projects/moscow_places.csv')
except: 
    data = pd.read_csv('/datasets/moscow_places.csv')

In [ ]:
#проверяем, считались ли данные 

data.head()

In [ ]:
#общая информация о датасете

data.info()

Датасет содержит 14 столбцов и 8 406 строк. Датасет содержит пропущенные значения.

6 столбцов содержит информацию в формате чисел с плавающей точкой (ширина и долгота заведения, рейтинг, средний чек, средняя цена кружки капучино, количество мест)

1 столбец содержит информацию в формате целых чисел (показатель того, сетевое заведение или нет)

7 столбцов содержат информацию в формате строк (наименование, категория заведения, адрес, район, часы работы, категория цен, средний чек)

[Вернуться наверх](#back)

<a id='solution_2'></a>

#### Шаг 2. Выполнить предобработку данных;

In [ ]:
#проверим уникальные значения столбцов - вдруг какие-то наименования внесены некорректно (поиск неявных дубликатов)

def chech_value_counts(data):
    for name in data.columns:
        print(data[name].value_counts())
        print('--------')

In [ ]:
chech_value_counts(data)

Нас интересуют столбцы: категория заведения, район, категория цен, сетевой ресторан или нет. Все значения написаны корректно.

In [ ]:
#поиск пропущенных значений

data_is_null = pd.DataFrame(data.isnull().sum())
data_is_null = data_is_null.assign(share = lambda x: round(x *100/ len(data),2))

data_is_null = data_is_null.rename(columns = {0: 'is_null'})

data_is_null

In [ ]:
data.columns

В датасете пропущенные значения содержатся в столбцах 

* график работы - вполне возможно, что в открытых данных пропущена эта информация, из-за чего они не попали в датасет (например, владелец заведения не указал график работы заведения в интернете). Пропущенные данные составляют 6% общего датасета. Расмотрим, нет ли каких-то технических причин данных пустот ниже;

* категория цен - вполне возможно, что в открытых данных пропущена эта информация, из-за чего они не попали в датасет. Пропущенные данные составляют 60% общего датасета. Расмотрим, нет ли каких-то технических причин данных пустот ниже;

* средний чек в виде диапазона и в виде числа - вполне возможно, что в открытых данных пропущена эта информация, из-за чего они не попали в датасет. Только доли у них отличаются между собой. Гипотеза: пропуски для значений среднего в целом числе связаны либо с отсутвием среднего в диапазоне, либо с тем, что заведение - кофейня или бар (пропуски будут характеры для цены кружки кофе или кружки пива). Ниже были проверена гипотеза - пропуски в столбце middle_coffee_cup связаны либо с отсутвием значений по среднему чеку, либо с категорией (кофейня или бар/паб).

* средняя цена кружки кофе - пропуски возможно связаны с категорией заведений - значения будут указаны только для кофеен. Проверим гипотезу ниже. Ниже проверена гипотеза: цена кружки указана для всех категорий заведений (не только для кофеен), при этом пропуски в данной категории присутствуют у 63% кофеен, что безусловно повлияет на результаты анализа. 

* количество мест - так же пропуски могут быть связаны не с технической причиной, а с отсутсвием данных по количеству мест в открытых источниках.

Таким образом, не было обнаружено каких-то пропусков, вызванных техническими проблемами. В основном пропуски могут быть связаны с тем, что владелец заведения не указал в открытых источниках данную информацию.

In [ ]:
# пропуски в графиках работы заведений - пропуски характерны для всех категорий

data[data['hours'].isnull()]['category'].unique()

In [ ]:
# пропуски в категории цен заведений - пропуски характерны для всех категорий

data[data['price'].isnull()]['category'].unique()

In [ ]:
# пропуски в среднем чеке заведений - пропуски характерны для категорий кофейня или бар/паб

new_dt_middle_avg_bill = data[data['middle_avg_bill'].isnull()]['avg_bill'].str.split(pat = ':', expand = True)

new_dt_middle_avg_bill[0].value_counts()

In [ ]:
# в диапазонах (avg_bill) - количество упоминананий "Средний счёт"  совпадает с ненулевыми значениями среднего чека в цифре (middle_avg_bill)

new_dt_avg_bill = data[data['avg_bill'].notnull()]['avg_bill'].str.split(pat = ':', expand = True)

new_dt_avg_bill[0].value_counts()

In [ ]:
# пропуски в middle_coffee_cup - было обнаружено, что цена чашки капучино указано не только для кофейни.

data[data['middle_coffee_cup'].notnull()]['category'].value_counts()

In [ ]:
# проверим, для всех ли кофеен указана цена чашки кофе

only_coffee = data[data['category'] == 'кофейня'][['name', 'middle_coffee_cup']]
 
print('Всего кофеен:', len(only_coffee))
print('Всего кофеен без указания средней цены за кофе:', only_coffee['middle_coffee_cup'].isna().sum(), 'или ',  round(only_coffee['middle_coffee_cup'].isna().sum()*100/len(only_coffee),2), '%')

63.13% пропущенных значений в категории кофейни, что усложнит дальнейший анализ лучшего места для открытия кофейни и среднюю стоимость кружки кофе там. Далее при анализе рассмотрим, как пустоты повлияют на результат анализа. 

In [ ]:
only_null_seats = data[data['seats'].isna()]
only_null_seats.groupby('category')['name'].count().sort_values(ascending = False)

Пропущенные значения встречаются во всех категориях заведений, поэтому пропуски скорее всего связаны с человеческим фактором (данных нет в окрытом источнике)

In [ ]:
#Поиск явных дубликатов - нет явных дубликатов

data.duplicated().sum()

In [ ]:
#рассмотрим аномальные значения в датасете (сделаем это только для тех столбцов, которые указаны в формате чисел):

data.info()

In [ ]:
# в столбце с широтой аномальных значений нет

data['lat'].describe()

In [ ]:
# в столбце с длинной аномальных значений нет

data['lng'].describe()

In [ ]:
# в столбце с рейтингом аномальных значений нет

data['rating'].describe()

In [ ]:
# в столбце с средним чеком были обнаружены аномальные значения

data['middle_avg_bill'].describe()

In [ ]:
#рассчитаем, какой средний чек не превышает 95% заведений Москвы

data['middle_avg_bill'].quantile(0.95)

5% заведений Москвы имеют средний чек, который превышает 2 250 руб. Далее в анализе обратим внимание на наличие аномальных значений в данном столбце.

In [ ]:
# в столбце с средней стоимостью кружки капучино были обнаружены аномальные значения

data['middle_coffee_cup'].describe()

In [ ]:
#рассчитаем, какая стоимость кружки капучино не превышает 95% заведений Москвы

data['middle_coffee_cup'].quantile(0.95)

5% заведений Москвы имеют стоимость кружки капучино, который превышает 275 руб. Далее в анализе обратим внимание на наличие аномальных значений в данном столбце

In [ ]:
# в столбце с количеством мест присутствуют аномальные значения

data['seats'].describe()

In [ ]:
data['seats'].quantile(0.95)

In [ ]:
data[data['seats'] > 1000].head()

Были обнаружены странные значения количества мест для заведений Москвы: для 95% заведений количество мест не превышает 307 мест, однако встречаются заведения с количеством мест более 1000 (и они встречаютмся на одной улице - Измайловское шоссе и проспект Вернадского). И их количество совпадает. Предположительно. это какая-то техническкая ошибка при выгрузке значений - подозрительное одинаковое значение количества мест заведений на одной улице и они в целом превышают 1000 мест.

In [ ]:
#Создайте столбец street с названиями улиц из столбца с адресом.

data['street'] = data['address'].str.split(pat = ',', expand = True)[1]

data.head()

In [ ]:
#Создайте столбец is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно (24/7):

data['is_24/7'] = data['hours'] == 'ежедневно, круглосуточно'

data.head()

### Выводы

Датасет содержит 14 столбцов и 8 406 строк. Датасет содержит пропущенные значения.

* 6 столбцов содержит информацию в формате чисел с плавающей точкой (ширина и долгота заведения, рейтинг, средний чек, средняя цена кружки капучино, количество мест)

* 1 столбец содержит информацию в формате целых чисел (показатель того, сетевое заведение или нет)

* 7 столбцов содержат информацию в формате строк (наименование, категория заведения, адрес, район, часы работы, категория цен, средний чек)

В процессе предобработки данных было выполнено:

1. Была сделана проверка на явные дубликаты - *нет явных дубликатов*

2. Была сделана проверка на неявные дубликаты - *нет неявных дубликатов*

3. Был выполнен анализ пропущенных значений: 

    * график работы - вполне возможно, что в открытых данных пропущена эта информация, из-за чего они не попали в датасет (например, владелец заведения не указал график работы заведения в интернете). Пропущенные данные составляют 6% общего датасета;
    
    * категория цен - вполне возможно, что в открытых данных пропущена эта информация, из-за чего они не попали в датасет. Пропущенные данные составляют 60% общего датасета. 
    
    * средний чек в виде диапазона и в виде числа - вполне возможно, что в открытых данных пропущена эта информация, из-за чего они не попали в датасет. Только доли у них отличаются между собой. Была проверена гипотеза - пропуски в столбце middle_coffee_cup связаны либо с отсутвием значений по среднему чеку, либо с категорией (кофейня или бар/паб).
    
    * средняя цена кружки кофе - пропуски возможно связаны с категорией заведений - значения будут указаны только для кофеен. Ниже проверена гипотеза: цена кружки указана для всех категорий заведений (не только для кофеен), при этом пропуски в данной категории присутствуют у 63% кофеен, что безусловно повлияет на результаты анализа рынка кофеен в Москве.
    
    * количество мест - так же пропуски могут быть связаны не с технической причиной, а с отсутсвием данных по количеству мест в открытых источниках.

Таким образом, не было обнаружено каких-то пропусков, вызванных техническими проблемами. В основном пропуски могут быть связаны с тем, что владелец заведения не указал в открытых источниках данную информацию.

4. Была выполнена проверка на аномальные значения. Были обнаружены аномальные значения в столбцах:

    *  *средний чек:* 5% заведений Москвы имеют средний чек, который превышает 2 250 руб. Далее в анализе обратим внимание на наличие аномальных значений в данном столбце;

    *  *средняя цена кружки капучино:* 5% заведений Москвы имеют стоимость кружки капучино, который превышает 275 руб. Далее в анализе обратим внимание на наличие аномальных значений в данном столбце;
    
    * *количество мест:* были обнаружены странные значения количества мест для заведений Москвы: для 95% заведений количество мест не превышает 307 мест, однако встречаются заведения с количеством мест более 1000 (и они встречаютмся на одной улице - Измайловское шоссе и проспект Вернадского). И их количество совпадает. Предположительно. это какая-то техническая ошибка при выгрузке значений - подозрительное одинаковое значение количества мест заведений на одной улице и они в целом превышают 1000 мест.


5. Были добавлены столбцы 'street' с названиями улиц и 'is_24/7' с обозначением, что заведение работает ежедневно и круглосуточно (24/7)

[Вернуться наверх](#back)

<a id='solution_3'></a>

#### Шаг 3. Провести анализ предоставленных данных;

In [ ]:
#Какие категории заведений представлены в данных? 
#Исследуйте количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее. 
#Постройте визуализации. Ответьте на вопрос о распределении заведений по категориям.

place_by_category = data.groupby('category')['name'].count()

fig = go.Figure(data=[go.Pie(labels = place_by_category.index, values=place_by_category)])

fig.update_layout(
    title="Всего заведений в Москве - 8 406")
fig.show() 

Наибольшую долю среди всех заведений Москвы занимают кафе - они составляют 28,3% всех заведений или 2 378 мест.
Затем идут рестораны, которые занимают 24,3% заведений или 2 043 мест.
Замыкают тройку лидеров кофейни, в Москве их 16,8% или 1 413 заведений.

Меньше всего в Москве заведений быстрого питания, столовых и булочных - вместе они занимают около 14% (603, 315 и 256 заведений соответственно).

Всего 8 категорий заведений.  


In [ ]:
#Исследуйте количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее. 
#Постройте визуализации. Проанализируйте результаты и сделайте выводы.

#Как раньше уже было отмечено, данный столбец содержит пустые значения и аномальные значения, поэтому для целей анализа количества мест по заведениям избавимся от них

places_by_seats = data[data['seats'] <= 307.0]

places_by_seats.head()

In [ ]:
#Из всего датасета оставшиеся строки составляют 54%

len(places_by_seats)/len(data) 

In [ ]:
#первый график - распределение количества мест по всем категориям 

#строим график
fig = px.histogram(data[data['seats'] <= 307], x='seats', title='В среднем в заведении около 40-60 мест', nbins = 20)

fig.update_xaxes(tickangle=45)

fig.update_layout(xaxis_title='количество заведений',
                   yaxis_title='количество мест')

fig.show() 

In [ ]:
seats_by_category = pd.DataFrame(data.groupby('category')['seats'].median()).reset_index() \
                     .rename(columns = {'seats': 'cnt'}) \
                     .sort_values(by = 'cnt', ascending = False)

#строим график
fig = px.bar(seats_by_category, x='category', y='cnt', title='Больше всего посадочных мест в ресторане - более 80')

fig.update_xaxes(tickangle=45)

fig.update_layout(xaxis_title='категория',
                   yaxis_title='количество мест')

fig.show() 

Для большинства заведений характерно примерно 50 посадочных мест - около 1000 заведений имеют примерно такое количество мест.

В среднем больше всего посадочных мест у ресторанов - 86 мест, затем идут бары и пабы - 83 мест, а на 3м месте - кофейни со средним количеством мест примерно 80.

Меньше всего мест в булочных - примерно 50 мест.

In [ ]:
#Рассмотрите и изобразите соотношение сетевых и несетевых заведений в датасете. Каких заведений больше?

#отобразим количество сетевых и несетевых заведений

places_chain = data.groupby('chain')['chain'].count().rename(index = {0: 'не сеть', 1: 'сеть'})

# строим диаграмму с сегментами
fig = go.Figure(data=[go.Pie(labels = places_chain.index,
                             values = places_chain, # указываем данные, которые отобразятся на графике
                             pull = [0.1, 0])]) # добавляем аргумент, который выделит сегмент-лидер на графике

fig.update_layout(title='Число сетевых и несетевых заведений', # указываем заголовок графика
                  width=700, # указываем размеры графика
                  height=500)

fig.show() # выводим график

Больше всего несетевых заведений - это 61.9% всех заведений или 5 201 заведение;

Сетевых заведений - 38.1% или 3 205 заведений

In [ ]:
places_by_category_chain = pd.pivot_table(data, index = 'category', values = ['name', 'chain'], aggfunc = {'name': 'count', 'chain':'sum'})

places_by_category_chain['share'] = round(places_by_category_chain['chain'] *100/ places_by_category_chain['name'],2)

places_by_category_chain = places_by_category_chain.sort_values(by = 'share', ascending = False).reset_index()

places_by_category_chain

In [ ]:
fig = px.bar(places_by_category_chain, x='category', y='share', title='Больше всего сетевых заведений среди булочных')

fig.update_xaxes(tickangle=45)

fig.update_layout(xaxis_title='категория',
                   yaxis_title='доля')

fig.show() 

Среди булочных, пиццерий и кофеен самая большая доля сетевых заведений - более 50%.
Меньше всего сетевых заведений среди баров и пабов - 22%.

In [ ]:
#Сгруппируйте данные по названиям заведений и найдите топ-15 популярных сетей в Москве. 
#Под популярностью понимается количество заведений этой сети в регионе. 
#Постройте подходящую для такой информации визуализацию. Знакомы ли вам эти сети? Есть ли какой-то признак, 
#который их объединяет? К какой категории заведений они относятся?

only_chain_places = data[data['chain'] == True]

only_chain_places.head()

In [ ]:
#самые популярные сети

#делаем датафрейм с топ 15 сетей
only_chain_places_top15 = pd.DataFrame(only_chain_places.groupby(['name', 'category'])['name'].count().sort_values(ascending = False).head(15)) #.rename(columns = {''})

#переименовываем и меняем индекс
only_chain_places_top15 = only_chain_places_top15.rename(columns = {'name': 'cnt'}).reset_index()

#выводим топ 15 сетей
display(only_chain_places_top15)

#посчитаем % заведений от всех заведений
print('Топ 15 сетей составляют', round(only_chain_places_top15['cnt'].sum()*100/len(data),2), '% заведений')

#строим график
fig = px.bar(only_chain_places_top15, x='name', y='cnt', title='Топ-15 сетей Москвы', color = 'category')

fig.update_xaxes(tickangle=45)

fig.update_layout(xaxis_title='наименование',
                   yaxis_title='количество мест',
                  legend_title_text='категория')

fig.show() 

Среди топ 15 сетей Москвы представлены:
* 6 кофеен;
* 2 пиццерии;
* 3 ресторана;
* 3 кафе;
* 1 булочная.

Топ-3 заведения по количеству заведений - кофейня Шоколадница, пиццерия Домино'с Пицца и Додо Пицца. Им соответствуют 119, 76 и 74 заведений соответственно.

В совокупности топ 15 сетей занимают 9.14% от всех заведений Москвы.

Популярность данных заведений может быть объяснена тем, что данные заведения можно открыть по франшизе: человек может приобрести за фиксированную сумму концепт заведения, имеющего бренд, клиентов и лояльность на рынке, и начать свой бизнес. Все лидеры категорий (Шоколадница в сфере кофеен, Домино в сфере пиццерий, Лавка в ресторанах, Кулинарная лавка братьев Караваевых в сфере кафе и Буханка в булочных) - это достаточно популярные франшизы Москвы. 

In [ ]:
#найдем категорию цен сетевых заведений

pd.DataFrame(only_chain_places[only_chain_places['name'].isin(only_chain_places_top15['name'])].groupby(['name', 'price'])['price'].count())

В основном категория цен у данных заведений - средние цены, выше среднего и низкие. Только у Кофемании встречаются высокие цены.

In [ ]:
# найдем медианный рейтинг сетевых заведений 

only_chain_places[only_chain_places['name'].isin(only_chain_places_top15['name'])].groupby(['name'])['name', 'rating'].median().sort_values(by = 'rating', ascending = False)

In [ ]:
# строим scatter
fig = px.box(only_chain_places[only_chain_places['name'].isin(only_chain_places_top15['name'])],             # загружаем данные
                 x='name', # указываем столбец с данными для оси X
                 y="rating",  # указываем столбец с данными для оси Y
                 title='Рейтинг сетевых заведений') # обозначаем категорию для разделения цветом

fig.update_layout(xaxis_title='наименование',
                   yaxis_title='рейтинг')

fig.show() 

У топ 15 сетевых заведений Москвы рейтинг равен или выше 4.0. Наивысший рейтинг имеют заведения Буханка - сеть маленьких пекарен возле дома - 4.4 балла из 5.

Наибольший разброс оценок наблюдается у Яндекс Лавки: при медианном значении 4.2, оценки в основном располагаются в диапазоне от 3.775 до 4.5. Наименьший разброс оценок - у Додо Пиццы.

In [ ]:
#Какие административные районы Москвы присутствуют в датасете? Отобразите общее количество заведений и 
#количество заведений каждой категории по районам. Попробуйте проиллюстрировать эту информацию одним графиком.

#сгруппировали данные по району и категории
district_places = data.groupby(['district', 'category'])['name'].count().reset_index()

#сгруппировали данные только по району
total_am_by_district = pd.DataFrame(district_places.groupby('district')['name'].sum().reset_index())

#объединим данные датафреймы и переименуем столбцы
total_am_by_district = total_am_by_district.merge(district_places, how = 'inner', on = 'district').rename(columns = {'name_x': 'total_amt', 'name_y': 'amt_by_category'})

#отсортируем датафрейм по общему количеству заведений, а затем по столбцу со значением по категориям
total_am_by_district = total_am_by_district.sort_values(by = ['total_amt', 'amt_by_category'], ascending = [False, False])

In [ ]:
#строим график
fig = px.bar(total_am_by_district, x='district', y='amt_by_category', title='Количество заведений по районам', color = 'category')

fig.update_xaxes(tickangle=45)

fig.update_layout(xaxis_title='район',
                   yaxis_title='количество мест',
                  legend_title_text='категория')
fig.show() 

Как можно заметить на графике выше, наибольшее количество заведений находится в центральном административном округе - превышение количества заведений более чем в 2 раза по сравдению с остальными районами. В абсолютном значении по категориям также больше всего по категориям заведений именно в центральном АО

In [ ]:
#Визуализируйте распределение средних рейтингов по категориям заведений. 
#Сильно ли различаются усреднённые рейтинги в разных типах общепита?

# строим scatter для расчета рейтингов по категориям заведений

fig = px.box(data,             # загружаем данные
             x='category', # указываем столбец с данными для оси X
             y="rating",  # указываем столбец с данными для оси Y
             title='Рейтинг заведений Москвы') # обозначаем категорию для разделения цветом

fig.update_layout(xaxis_title='категория',
                   yaxis_title='рейтинг')

fig.show() 

В целом по заведениям медианный рейтинг превышает 4.0, большинство заведений имеют рейтинг от 4 до 4.5, кроме заведений быстрого питания - для них рейтинг в основном от 3.9 до 4.3

In [ ]:
#Постройте фоновую картограмму (хороплет) со средним рейтингом заведений каждого района. 

data.groupby('district')['rating'].mean()

In [ ]:
# читаем файл и сохраняем в переменной
try:
    with open('/datasets/admin_level_geomap.geojson', 'r') as f:
        geo_json = json.load(f)
except:
    with open('/Users/alina/Desktop/projects/admin_level_geomap.geojson', 'r') as f:
        geo_json = json.load(f)    

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'
    
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

In [ ]:
state_geo

In [ ]:
# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту

Choropleth(geo_data=state_geo,
    data=data.groupby('district')['rating'].mean(),
    columns=['rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

В центральном районе находится самое большое количество заведений, при этом они обладают наибольшим рейтингом по сравнению с другими регионами Москвы. Также высоким рейтингом (по сравнению с другими регионами) обладает Северный административный округ и Северо-Западный административный округ. Самым низким рейтингом обладает Юго-Восточный административный округ.

In [ ]:
#Отобразите все заведения датасета на карте с помощью кластеров средствами библиотеки folium.

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
        tooltip = row['category']
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.apply(create_clusters, axis=1)

# выводим карту
m

Выше на карте Москвы были отражены все заведения. При наведении на заведение автоматически появляется информация о категории заведения, а при нажатии - информация о наименовании заведения и его рейтинг

In [ ]:
#Найдите топ-15 улиц по количеству заведений. Постройте график распределения количества заведений и их категорий по этим улицам.
#Попробуйте проиллюстрировать эту информацию одним графиком.

#найдем топ 15 улиц Москвы по количеству заведений
places_by_street = data.groupby('street')['name'].count().sort_values(ascending = False).head(15)

display(pd.DataFrame(places_by_street))

In [ ]:
#отберем из датафрейма данные только по топ 15 улицам
places_by_street_and_category = data[data['street'].isin(places_by_street.index)][['name', 'category', 'street']]

#сгруппируем данные по улице, категории
places_by_street_and_category = pd.DataFrame(places_by_street_and_category.groupby(['street', 'category'])['name'].count()).reset_index()


places_by_street_and_category.sort_values(by = ['street', 'name'], ascending = [True, False])

places_by_street_and_category

In [ ]:
places_by_street_and_category = pd.DataFrame(places_by_street_and_category.groupby('street')['name'].sum()) \
                                .merge(places_by_street_and_category, how = 'inner', on = 'street')

places_by_street_and_category_sorted = places_by_street_and_category.rename(columns = {'name_x': 'total_amt', 'name_y': 'amt_by_category'}) \
                                .sort_values(by = ['total_amt', 'amt_by_category'], ascending = [False, False])

places_by_street_and_category_sorted

In [ ]:
#строим график
fig = px.bar(places_by_street_and_category_sorted, \
             x='street', \
             y='amt_by_category', title='Количество заведений на топ-15 улицах', color = 'category')

fig.update_xaxes(tickangle=45)

fig.update_layout(xaxis_title='улица',
                   yaxis_title='количество мест',
                  legend_title_text='категория')
fig.show() 

Больше всего заведений на Проспекте Мира - всего 184 заведения. Затем идут Профсоюзная улица (122 заведения) и проспект Вернадского (108 заведений).
В целом по категориям на топ 15 улицах преобладают рестораны и кафе (доли этих заведений на графике максимальны)

In [ ]:
#Найдите улицы, на которых находится только один объект общепита. Что можно сказать об этих заведениях?

places_by_street = pd.DataFrame(data.groupby('street')['name'].count()).reset_index().rename(columns = {'name': 'cnt'})

streets_with_one_place = places_by_street[places_by_street['cnt'] == 1]

streets_with_one_place

В Москве всего 458 улиц, на которых располагаются только одно заведение. Рассмотрим более подробно их

In [ ]:
streets_with_one_place_data = data[data['street'].isin(streets_with_one_place['street'])]

streets_with_one_place_data.head()

In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
        tooltip = row['category']
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
streets_with_one_place_data.apply(create_clusters, axis=1)

# выводим карту
m

Все заведения располагаются на всей территории карты, то есть нельзя сказать, что заведения располагаются только на окраинах города. Даже оказалось, что большинство таких улиц располагаются в центре города. Скорее всего такие улицы очень маленькие (одно заведение как раз удовлетворит потребности целой улицы) или отсутсвует информация о наличии там дополнительных заведений

In [ ]:
streets_with_one_place_data_group = streets_with_one_place_data.groupby(['category']).agg({'name':'count', 'rating': 'median'}).reset_index().sort_values(by = 'name', ascending = False)

streets_with_one_place_data_group
#строим график

fig = px.bar(streets_with_one_place_data_group, x='category', y='name', \
             title='Количество заведений-одиночек на улицах Москвы', color = 'rating')

fig.update_xaxes(tickangle=45)

fig.update_layout(xaxis_title='категория',
                   yaxis_title='количество заведений')

fig.show() 

In [ ]:
data.merge(data.groupby('street')['name'].count().rename('is_alone') == 1, on='street')\
    .groupby('is_alone')[['seats','rating','middle_avg_bill']].mean()

Для заведений-одиночек характерно в среднем меньшее количество посадочных мест, незначительно выше средний рейтинг и незначительно выше средний чек

Наибольшее количество заведений-одиночек на улицах Москвы являются кафе (160 штук), при этом именно кафе имеют наименьший медианный рейтинг заведений - 4.2.

Наименьшее количество заведений-одиночек - булочные (8 заведений), при этом они имеют медианный рейтинг 4.35-

Наибольший рейтинг среди заведений-одиночек имеют бары и пабы - 4.5 (всего 39 заведений)

In [ ]:
#Посчитайте медиану столбца middle_avg_bill для каждого района. 
#Используйте это значение в качестве ценового индикатора района. Постройте фоновую картограмму (хороплет) 
#с полученными значениями для каждого района. Проанализируйте цены в центральном административном округе и других. 
#Как удалённость от центра влияет на цены в заведениях?

#для начала отберем те значения middle_avg_bill, характерные для 95% всех заведений (без учета аномальных значений) - средний чек составляет не более 2250 руб.

data['middle_avg_bill'].quantile(0.95)

In [ ]:
middle_price_by_district = data[(data['middle_avg_bill'] < 2250) | (data['middle_avg_bill'].isna()) ]

middle_price_by_district.head()

In [ ]:
middle_price_by_district_grouped = middle_price_by_district.groupby('district') \
                                   .agg({'middle_avg_bill': 'median'})

middle_price_by_district_grouped

In [ ]:
# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=middle_price_by_district_grouped.squeeze(),
    columns=['rating'],
    key_on='feature.name',
    fill_color='Reds',
    fill_opacity=0.8,
    legend_name='Медианный средний чек заведения по районам',
).add_to(m)

# выводим карту
m

Как видно на карте, наибольший средний чек наблюдается в заведениях, расположенных в центральном АО - 905 руб.;

Затем идут заведения, расположенные на западной стороне города: Западный административный округ (750 руб.) и Северо-Западный административный округ (675 руб.)

Наименьший средний чек наблюдается у восточных районов Москвы: Восточный административный округ	и Южный АО (по 500 руб.)

In [ ]:
#рассмотрим как будет выглядить распределение среднего чека только категории "Кафе"

middle_price_by_district_grouped_2 = middle_price_by_district[middle_price_by_district['category'] == 'кафе'].groupby('district') \
                                   .agg({'middle_avg_bill': 'median'})

middle_price_by_district_grouped_2

In [ ]:
# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=middle_price_by_district_grouped_2.squeeze(),
    columns=['rating'],
    key_on='feature.name',
    fill_color='Reds',
    fill_opacity=0.8,
    legend_name='Медианный средний чек заведения по районам',
).add_to(m)

# выводим карту
m

Центальный район все еще в топе по среднему чеку по заведениям (кафе) в Москве. Затем идут западные АО (Западный АО, Северо-западный АО) и Южный АО - 500 руб.

Самый низкий средний чек имеют заведения в Юго-Восточном АО - 375 руб.

Чем дальше из центра - тем ниже средний чек (что логично), при этом при движении на запад - цена снижается примерно на 15-20%, а при движении на восток - почти на 50% 

In [ ]:
#проиллюстрируйте другие взаимосвязи, которые вы нашли в данных. 
#Например, по желанию исследуйте часы работы заведений и их зависимость от расположения и категории заведения. 
#Также можно исследовать особенности заведений с плохими рейтингами, средние чеки в таких местах и распределение по 
#категориям заведений.

In [ ]:
#рассмотрим заведения с "плохим" рейтингом

data['rating'].describe()

В основном заведения получаюи оценки от 4 до 5, рассмотрим заведения, которые имеют оценки ниже 4.0 - всего 1 169 заведений

In [ ]:
low_rating_places = data[data['rating'] < 4.0]

display(low_rating_places.head())
print(len(low_rating_places))

In [ ]:
low_rating_places_cnt = low_rating_places.groupby('category')['name'].count().reset_index().sort_values(ascending = False, by = 'name')

#строим график

fig = px.bar(low_rating_places_cnt, x='category', y='name', \
             title='Количество заведений с низким рейтингом по категориям')

fig.update_xaxes(tickangle=45)

fig.update_layout(xaxis_title='категория',
                   yaxis_title='количество заведений')

fig.show() 

In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
        tooltip = row['category']
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
low_rating_places.apply(create_clusters, axis=1)

# выводим карту
m

Большинство заведений все еще располагаются ближе к центру города, однако разрыв по сравнению с другими районами не такой сильный. Преобладающее количество заведений с низким рейтингом находятся в центральном районе, на юго-западе и северо-западе Москвы.

Больше всего заведений с низким рейтингом - кафе - более 500 заведений, затем идут рестораны и заведения быстрого питания - примерно по 200 заведений. Меньше всего заведений с низким рейтингом - булочные - 30 заведений.

In [ ]:
#создаем датафрейм с распределением заведений по категориям
places_by_category = pd.DataFrame(data.groupby('category')['name'].count()).reset_index().rename(columns = {'name': 'cnt'})

#создаем датафрейм с распределением заведений по категориям только с низким рейтингом
places_by_category_low_rating = pd.DataFrame(low_rating_places.groupby('category')['name'].count()).reset_index().rename(columns = {'name': 'cnt_low_rating'})

#объединяем датафреймы
places_by_category_merged = places_by_category.merge(places_by_category_low_rating, how = 'left', left_on = 'category', right_on = 'category')

#рассмотрим относительную долю заведений с низким рейтингом от всех заведений по категориям
places_by_category_merged['share'] =  places_by_category_merged['cnt_low_rating'] / places_by_category_merged['cnt']

places_by_category_merged.sort_values(by = 'share', ascending = False)

Посчитав относительную долю заведений с плохими рейтингами по отношению ко всем заведениям, было выяснено, что больше всего заведений с низким рейтингом встречается среди заведений быстрого питания (27%), кафе (23%) и столовых (14%). Это может быть связано с низким качеством еды, предлагаемых в заведениях быстрого питания и столовых. Приоритетом в таких заведениях является скорость приготовления еды и низкая цена.

Меньше всего заведений с низким рейтигом встречается у баров и пабов (5% от всего количества)

In [ ]:
low_rating_places.groupby('chain')['name'].count()

Не сетевые заведения составляют все еще преобладающую долю среди сетевых заведений.

In [ ]:
#найдем топ-5 сетей по количеству заведений с низкими рейтигами

#создадим датафрейм с количеством заведений по сетевому признаку
places_by_chain = pd.DataFrame(only_chain_places.groupby('name')['name'].count()).rename(columns = {'name':'cnt'}).reset_index()

#создадим датафрейм с количеством заведений по сетевому признаку только с низким рейтингом (меньше 4)
places_by_chain_low_rating = pd.DataFrame(only_chain_places[only_chain_places['rating'] < 4.0].groupby('name')['name'].count()).rename(columns = {'name':'cnt_low_rating'}).reset_index()

#объединим данные датафреймы
places_by_chain = places_by_chain.merge(places_by_chain_low_rating, how = 'right', left_on = 'name', right_on = 'name')

#создадим столбец с долей заведений с низким рейтингом среди всех заведений
places_by_chain['share'] = places_by_chain['cnt_low_rating'] / places_by_chain['cnt']

#отсортируем по доле
places_by_chain = places_by_chain.sort_values(by = 'share', ascending = False)

#выведем топ 5 по относительной доли заведений с низким рейтингом
display(places_by_chain[places_by_chain['cnt'] > 1].head())
display(places_by_chain[places_by_chain['cnt'] > 1].tail())

5 сетей с наибольшим количеством заведениями с низкими рейтингами:
    
* DimSum & Co - 4 из 4 заведений;
* Karavan - 2 из 2 заведений;
* Восточная кухня - 5 из 5 заведений;
* Вояж - 3 из 3 заведений;
* Green V. A. I. - 2 из 2 заведений;

5 сетей с наименьшим количеством заведениями с низкими рейтингами:
    
* Штолле & Co - 1 из 19 заведений;
* АндерСон - 1 из 22 заведений;
* Буханка - 1 из 32 заведений;
* Шоколадница - 3 из 120 заведений;
* Додо Пицца - 1 из 74 заведений;

#### Выводы

На шаге №3 был проведен анализ предоставленных данных. Для этого были рассмотрены зависимости в данных по категориям:

* по типу заведения;
* по количеству посадочных мест;
* по соотношению сетевых и несетевых заведений;
* по самым популярным сетевым заведениям Москвы;
* по административным районам Москвы;
* по средним рейтингам заведений;
* по количеству заведений на определенных улицах (самые популярные улицы или улицы с заведениями-одиночками);
* среднему чеку заведений;
* по заведениям только с низким рейтингом.

Основные тезисы, которые были определены после анализа:

**Тип заведения**

Всего 8 категорий заведений: рестораны, кафе, бары и пабы, быстрое питание, булочные, кофейни, столовые, пиццерии.

Наибольшую долю среди всех заведений Москвы занимают кафе - они составляют 28,3% всех заведений или 2 378 мест. Затем идут рестораны, которые занимают 24,3% заведений или 2 043 мест. Замыкают тройку лидеров кофейни, в Москве их 16,8% или 1 413 заведений.Меньше всего в Москве заведений быстрого питания, столовых и булочных - вместе они занимают около 14% (603, 315 и 256 заведений соответственно).

**Количество посадочных мест**

Для большинства заведений характерно примерно 50 посадочных мест - около 1000 заведений имеют примерно такое количество мест.

В среднем больше всего посадочных мест у ресторанов - 86 мест, затем идут бары и пабы - 83 мест, а на 3м месте - кофейни со средним количеством мест примерно 80. Меньше всего мест в булочных - примерно 50 мест.

**Сеть/не сеть**

Больше всего несетевых заведений - это 61.9% всех заведений или 5 201 заведение;
Сетевых заведений - 38.1% или 3 205 заведений.

**Самые популярные сети**

Среди топ 15 сетей Москвы представлены:

* 6 кофеен;
* 2 пиццерии;
* 3 ресторана;
* 3 кафе;
* 1 булочная.

Топ-3 сетей по количеству заведений - кофейня Шоколадница, пиццерия Домино'с Пицца и Додо Пицца. Им соответствуют 119, 76 и 74 заведений соответственно.

В совокупности топ 15 сетей занимают 9.14% от всех заведений Москвы.

Популярность данных заведений может быть объяснена тем, что данные заведения можно открыть по франшизе: человек может приобрести за фиксированную сумму концепт заведения, имеющего бренд, клиентов и лояльность на рынке, и начать свой бизнес. Все лидеры категорий (Шоколадница в сфере кофеен, Домино в сфере пиццерий, Лавка в ресторанах, Кулинарная лавка братьев Караваевых в сфере кафе и Буханка в булочных) - это достаточно популярные франшизы Москвы.

В основном категория цен у данных заведений - средние цены, выше среднего и низкие. Только у Кофемании встречаются высокие цены.

У топ 15 сетевых заведений Москвы рейтинг равен или выше 4.0. Наивысший рейтинг имеют заведения Буханка - сеть маленьких пекарен возле дома - 4.4 балла из 5.

Наибольший разброс оценок наблюдается у Яндекс Лавки: при медианном значении 4.2, оценки в основном располагаются в диапазоне от 3.775 до 4.5. Наименьший разброс оценок - у Додо Пиццы.

**АО Москвы**

Наибольшее количество заведений находится в центральном административном округе - превышение количества заведений более чем в 2 раза по сравнению с остальными районами. В абсолютном значении по категориям также больше всего по категориям заведений именно в центральном АО.

**Средний рейтинг заведений**

В целом по заведениям медианный рейтинг превышает 4.0, большинство заведений имеют рейтинг от 4 до 4.5, кроме заведений быстрого питания - для них рейтинг в основном от 3.9 до 4.3.

В центральном районе находится самое большое количество заведений, при этом они обладают наибольшим рейтингом по сравнению с другими регионами Москвы. Также высоким рейтингом (по сравнению с другими регионами) обладает Северный административный округ и Северо-Западный административный округ. Самым низким рейтингом обладает Юго-Восточный административный округ.

**Самые популярные и непопулярые улицы**

Больше всего заведений на Проспекте Мира - всего 184 заведения. Затем идут Профсоюзная улица (122 заведения) и проспект Вернадского (108 заведений). В целом по категориям на топ 15 улицах преобладают рестораны и кафе (доли этих заведений на графике максимальны).

В Москве всего 458 улиц, на которых располагаются только одно заведение. Все заведения располагаются на всей территории карты, то есть нельзя сказать, что заведения располагаются только на окраинах города. Даже оказалось, что большинство таких улиц располагаются в центре города. Скорее всего такие улицы очень маленькие (одно заведение как раз удовлетворит потребности целой улицы) или отсутсвует информация о наличии там дополнительных заведений.

Наибольшее количество заведений-одиночек на улицах Москвы являются кафе (160 штук), при этом именно кафе имеют наименьший медианный рейтинг заведений - 4.2. Наименьшее количество заведений-одиночек - булочные (8 заведений), при этом они имеют медианный рейтинг 4.35.Наибольший рейтинг среди заведений-одиночек имеют бары и пабы - 4.5 (всего 39 заведений).

**Средний чек**

Наибольший средний чек наблюдается в заведениях, расположенных в центральном АО - 905 руб.; Затем идут заведения, расположенные на западной стороне города: Западный административный округ (750 руб.) и Северо-Западный административный округ (675 руб.). Наименьший средний чек наблюдается у восточных районов Москвы: Восточный административный округ и Южный АО (по 500 руб.).

Отобрав только одну категорию заведений - кафе - центальный район все еще в топе по среднему чеку по заведениям (кафе) в Москве. Затем идут западные АО (Западный АО, Северо-западный АО) и Южный АО - 500 руб. Самый низкий средний чек имеют заведения в Юго-Восточном АО - 375 руб.

Чем дальше из центра - тем ниже средний чек (что логично), при этом при движении на запад - цена снижается примерно на 15-20%, а при движении на восток - почти на 50%

Для заведений-одиночек характерно в среднем меньшее количество посадочных мест, незначительно выше средний рейтинг и незначительно выше средний чек.

**Заведения с низким рейтингом**

В основном заведения получают оценки от 4 до 5, рассмотрим заведения, которые имеют оценки ниже 4.0 - всего 1 169 заведений.

Большинство заведений все еще располагаются ближе к центру города, однако разрыв по сравнению с другими районами не такой сильный. Преобладающее количество заведений с низким рейтингом находятся в центральном районе, на юго-западе и северо-западе Москвы.

Больше всего заведений с низким рейтингом - кафе - более 500 заведений, затем идут рестораны и заведения быстрого питания - примерно по 200 заведений. Меньше всего заведений с низким рейтингом - булочные - 30 заведений.

Посчитав относительную долю заведений с плохими рейтингами по отношению ко всем заведениям, было выяснено, что больше всего заведений с низким рейтингом встречается среди заведений быстрого питания (27%), кафе (23%) и столовых (14%). Это может быть связано с низким качеством еды, предлагаемых в заведениях быстрого питания и столовых. Приоритетом в таких заведениях является скорость приготовления еды и низкая цена. Меньше всего заведений с низким рейтигом встречается у баров и пабов (5% от всего количества).

Не сетевые заведения составляют все еще преобладающую долю среди сетевых заведений.

5 сетей с наибольшим количеством заведениями с низкими рейтингами:

* DimSum & Co - 4 из 4 заведений;
* Karavan - 2 из 2 заведений;
* Восточная кухня - 5 из 5 заведений;
* Вояж - 3 из 3 заведений;
* Green V. A. I. - 2 из 2 заведений;

5 сетей с наименьшим количеством заведениями с низкими рейтингами:

* Штолле - 1 из 19 заведений;
* АндерСон - 1 из 22 заведений;
* Буханка - 1 из 32 заведений;
* Шоколадница - 3 из 120 заведений;
* Додо Пицца - 1 из 74 заведений;


[Вернуться наверх](#back)

<a id='solution_4'></a>

#### Шаг 4. Провести детализацию исследования: открытие кофейни

In [ ]:
#Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?

only_coffee_data = data[data['category'] == 'кофейня'] 

print(f'Всего кофеен в датасете: {len(only_coffee_data)}')

In [ ]:
only_coffee_data.groupby('district')['name'].count()

In [ ]:
# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=only_coffee_data.groupby('district')['name'].count(),
    key_on='feature.name',
    fill_color='Greys',
    fill_opacity=0.8,
    legend_name='Количество заведений по районам',
).add_to(m)

# выводим карту
m

Больше всего кофеен в центральном АО - 428 заведений.
С отставанием почти в 2 раза - Северный административный округ (193 заведений) и Северо-Восточный административный округ (159 заведений).
Меньше всего заведений на востоке и северо-западе города - Северо-Западный административный округ (менее 100 на АО).

In [ ]:
only_coffee_data.groupby('chain')['name'].count()

Среди кофеен практически одинаковое распределение сетевых и несетевых заведений.

In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
        tooltip = row['category']
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
only_coffee_data.apply(create_clusters, axis=1)

# выводим карту
m

In [ ]:
only_coffee_data.groupby('street')['name'].count().sort_values(ascending = False)

Выше на карте нанесены все кофейни Москвы - много кофеен расположено на территорий ТЦ (именно там самый большой поток людей). Формат таких заведений предполагает наличие небольшого пространсва, что подходит для расположения в ТЦ.


По количеству заведений лидируют улицы "Проспект Мира" (36 заведений), "Ленинградский проспект" (25 заведений), "Ленинский проспект" (23 заведения).

В основном целевая аудитория кофеен зависит от расположения кофейни - близость к БЦ привнесет основную категорию клиентов в виде работников БЦ с предпочтением "кофе с собой". Островок в ТЦ также привлечет клиентов в виде проходящих мимо посетителей ТЦ, предпочитающих взять кофе с собой и пройти дальше за покупками.

Концепт кофейни в формате друзей предполагает место встречи друзей для длительных посиделок в компании за разговорами. Концепт такого заведения: кофейня, похожая на кафе: больше посадочных мест мес, уютная атмосфера, вкусный кофе. Хорошо, если заведение будет располагаться близко к метро (так удобнее будет добираться всем друзьям"). При этом не обязательно кофейня должна располагаться близко к центру: так можно сохранить уют и теплоту заведения. 

In [ ]:
#Есть ли круглосуточные кофейни?

display(only_coffee_data.groupby('is_24/7')['name'].count())

print(only_coffee_data['is_24/7'].sum()/len(only_coffee_data))

Среди заведений есть круглосуточные кофейни, работающие без выходных, но они составляют всего 4% заведений. В основном заведения работают ежедневно, с 10:00–22:00 (140 заведений), ежедневно, с 08:00–22:00 (60 заведений)

In [ ]:
only_coffee_data.groupby('hours')['name'].count().sort_values(ascending = False)

In [ ]:
#Какие у кофеен рейтинги? Как они распределяются по районам?

# строим scatter для расчета рейтингов по категориям заведений

fig = px.box(only_coffee_data,             # загружаем данные
             x = 'chain', 
             y="rating",  # указываем столбец с данными для оси Y
             title='Рейтинг кофеен Москвы') # обозначаем категорию для разделения цветом

fig.update_layout(xaxis_title='сеть/не сеть',
                   yaxis_title='рейтинг')

fig.show() 

Медианный рейтинг по несетевым заведениям - 4.4, а рейтинг сетевых заведений при этом ниже - 4.2

Максимальный рейтинг по несетевым заведениям и сетевым заведениям - 5, минимальный - 1.4 и 2 соответственно


In [ ]:
# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=only_coffee_data.groupby('district')['rating'].median(),
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг кофейни по районам',
).add_to(m)

# выводим карту
m

Было обнаружено, что в основном все районы Москвы имеют достаточно высокий рейтинг - от 4.2 до 4.3, при этом только в Западном административном округе медианный рейтинг ниже всего - 4.2 

In [ ]:
only_coffee_data.groupby('district')['rating'].median()

In [ ]:
#На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

#Посмотрим среднюю стоимость чашки капучино по районам

In [ ]:
only_coffee_data.groupby('district')['middle_coffee_cup'].median()

In [ ]:
# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=only_coffee_data.groupby('district')['middle_coffee_cup'].median(),
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.8,
    legend_name='Медианный средний чек кофейни по районам',
).add_to(m)

# выводим карту
m

Самые высокие медианные значения кружки капучино в районах Москвы расположены в Центральном АО, Юго-Западном АО и Западном АО - 190 руб, 198 руб и 189 руб соответственно.

Самое низкое значение медианной стоимости кружки кофе в Восточный АО - 135 руб

С точки зрения района открытия кофейни именно Западный АО выглядит привлекательно:
    
1. В данном районе около 150 кофеен, при этом это не самое большое количество кофеен на АО - район выглядит не переполненнным кофейнями;

2. Кофейни в данном районе имеют рейтинг ниже по сравнению с кофейнями в других АО - открытие классной кофейни могло бы переманить клиентов; 

3. Медианная стоимость кружки кофе - одна из максимальных среди всех АО, что говорит о том, что клиентура достаточно платежеспособна.

Отберем 5 станции метро, расположенное в ЗАО, недалеко от центра: Киевская, Парк Победы, Студенческая, Университет, Ломоносовский проспект

In [ ]:
#4 станции метро: широта и долгота

# сохраняем координаты метро Киевская в переменные
kievskaya_lat, kievskaya_lng = 55.74440199487915, 37.56547625442412

# сохраняем координаты метро Парк Победы в переменные
park_lat, park_lng = 55.73621890017415,37.51802967303838

# сохраняем координаты метро Студенческая в переменные
studencheskaya_lat, studencheskaya_lng = 55.739366, 37.545707

# сохраняем координаты метро Университет в переменные
universitet_lat, universitet_lng = 55.692222, 37.532778

# сохраняем координаты метро Ломоносовский проспект в переменные
lp_lat, lp_lng = 55.70715284384157,37.5163614563581

# общий тип мест
place_type = 'Кофейня'

# создаём карту с центром в точке расположения метро Парк Победы и начальным зумом 12
m = folium.Map(location=[park_lat, park_lng], zoom_start=12)

icon_url = 'https://img.icons8.com/ios-filled/512/coffee.png'

icon1 = CustomIcon(icon_url, icon_size=(30, 30))
icon2 = CustomIcon(icon_url, icon_size=(30, 30))
icon3 = CustomIcon(icon_url, icon_size=(30, 30))
icon4 = CustomIcon(icon_url, icon_size=(30, 30))
icon5 = CustomIcon(icon_url, icon_size=(30, 30))

# создаём маркер в точке расположения метро Киевская и сразу добавляем на карту
folium.Marker([kievskaya_lat, kievskaya_lng], tooltip=place_type,\
              popup='Киевская', \
             icon = icon1).add_to(m)

# создаём маркер в точке расположения метро Парк Победы и сразу добавляем на карту
folium.Marker([park_lat, park_lng], tooltip=place_type,\
              popup='Парк Победы', \
             icon = icon2).add_to(m)

# создаём маркер в точке расположения метро Студенческая и сразу добавляем на карту
folium.Marker([studencheskaya_lat, studencheskaya_lng], tooltip=place_type,\
              popup='Студенческая', \
             icon = icon3).add_to(m)

# создаём маркер в точке расположения метро Университет и сразу добавляем на карту
folium.Marker([universitet_lat, universitet_lng], tooltip=place_type,\
              popup='Университет', \
             icon = icon4).add_to(m)

# создаём маркер в точке расположения метро Ломоносовский проспект и сразу добавляем на карту
folium.Marker([lp_lat, lp_lng], tooltip=place_type,\
              popup='Ломоносовский проспект', \
             icon = icon5).add_to(m)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# применяем функцию create_clusters() к каждой строке датафрейма
only_coffee_data.apply(create_clusters, axis=1)

# выводим карту
m

Задаем стандартные условия графика работы: ежедневно, 10:00–22:00. Данное время будет оптимальным для кофейни для долгих посиделок с друзьями.
            
Цена кружки капучино поставим на уровне медианного значения: 180-190 руб за кружку кофе. Так мы сможем обеспечить оплату аренды заведения у метро и поставить приемлимую цену за кофе по сравнению с другими кофейнями.            

#### Выводы:
    

Всего кофеен в датасете: 1413.
    
Больше всего кофеен в центральном АО - 428 заведений. С отставанием почти в 2 раза - Северный административный округ (193 заведений) и Северо-Восточный административный округ (159 заведений). Меньше всего заведений на востоке и северо-западе города - Северо-Западный административный округ (менее 100 на АО).

Среди кофеен практически одинаковое распределение сетевых и несетевых заведений.

Выше на карте нанесены все кофейни Москвы - много кофеен расположено на территорий ТЦ (именно там самый большой поток людей). Формат таких заведений предполагает наличие небольшого пространсва, что подходит для расположения в ТЦ.

По количеству заведений лидируют улицы "Проспект Мира" (36 заведений), "Ленинградский проспект" (25 заведений), "Ленинский проспект" (23 заведения).

В основном целевая аудитория кофеен зависит от расположения кофейни - близость к БЦ привнесет основную категорию клиентов в виде работников БЦ с предпочтением "кофе с собой". Островок в ТЦ также привлечет клиентов в виде проходящих мимо посетителей ТЦ, предпочитающих взять кофе с собой и пройти дальше за покупками.

Концепт кофейни в формате друзей предполагает место встречи друзей для длительных посиделок в компании за разговорами. Концепт такого заведения: кофейня, похожая на кафе: больше посадочных мест мес, уютная атмосфера, вкусный кофе. Хорошо, если заведение будет располагаться близко к метро (так удобнее будет добираться всем друзьям). При этом не обязательно кофейня должна располагаться близко к центру: так можно сохранить уют и теплоту заведения.

Среди заведений есть круглосуточные кофейни, работающие без выходных, но они составляют всего 4% заведений. В основном заведения работают ежедневно, с 10:00–22:00 (140 заведений), ежедневно, с 08:00–22:00 (60 заведений).

Медианный рейтинг по несетевым заведениям - 4.4, а рейтинг сетевых заведений при этом ниже - 4.2. Максимальный рейтинг по несетевым заведениям и сетевым заведениям - 5, минимальный - 1.4 и 2 соответственно.

Было обнаружено, что в основном все районы Москвы имеют достаточно высокий рейтинг - от 4.2 до 4.3, при этом только в Западном административном округе медианный рейтинг ниже всего - 4.2.

Самые высокие медианные значения кружки капучино в районах Москвы расположены в Центральном АО, Юго-Западном АО и Западном АО - 190 руб, 198 руб и 189 руб соответственно. Самое низкое значение медианной стоимости кружки кофе в Восточный АО - 135 руб.

С точки зрения района открытия кофейни именно Западный АО выглядит привлекательно:

* В данном районе около 150 кофеен, при этом это не самое большое количество кофеен на АО - район выглядит не переполненнным кофейнями;

* Кофейни в данном районе имеют рейтинг ниже по сравнению с кофейнями в других АО - открытие классной кофейни могло бы переманить клиентов;

* Медианная стоимость кружки кофе - одна из максимальных среди всех АО, что говорит о том, что клиентура достаточно платежеспособна.

Отберем 5 станции метро, расположенное в ЗАО, недалеко от центра: Киевская, Парк Победы, Студенческая, Университет, Ломоносовский проспект.

Задаем стандартные условия графика работы: ежедневно, 10:00–22:00. Данное время будет оптимальным для кофейни для долгих посиделок с друзьями.

Цена кружки капучино поставим на уровне медианного значения: 180-190 руб за кружку кофе. Так мы сможем обеспечить оплату аренды заведения у метро и поставить приемлимую цену за кофе по сравнению с другими кофейнями.

[Вернуться наверх](#back)